In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install trimesh


In [ ]:
import os

DATA_DIR = "/kaggle/input/mugs-normalised/mugs_normalized"
print(os.listdir(DATA_DIR))


In [ ]:
import trimesh

mesh = trimesh.load(f"{DATA_DIR}/mug1.obj")
mesh.show()


In [ ]:
import numpy as np

# I am sampling points in the bounding box of the mesh
bounds = mesh.bounds
points = np.random.uniform(
    low=bounds[0],
    high=bounds[1],
    size=(50000, 3)
)

points.shape


In [ ]:
sdf = trimesh.proximity.signed_distance(mesh, points)

sdf.shape


In [ ]:
import matplotlib.pyplot as plt

inside = sdf < 0
outside = sdf > 0

plt.figure(figsize=(6,6))
plt.scatter(points[outside][:,0], points[outside][:,1],
            s=1, c='red', label='outside')
plt.scatter(points[inside][:,0], points[inside][:,1],
            s=1, c='blue', label='inside')

plt.legend()
plt.axis('equal')
plt.title("Signed Distance Field Sampling")
plt.show()


In [ ]:
import os
import pickle

DATA_DIR = "/kaggle/input/mugs-normalised/mugs_normalized"

all_sdf = {}

for fname in os.listdir(DATA_DIR):
    if not fname.endswith(".obj"):
        continue   

    mesh = trimesh.load(os.path.join(DATA_DIR, fname))
    pts = np.random.uniform(mesh.bounds[0], mesh.bounds[1], (30000, 3))
    sdf_vals = trimesh.proximity.signed_distance(mesh, pts)

    all_sdf[fname] = {
        "points": pts,
        "sdf": sdf_vals
    }

print("Processed objects:", len(all_sdf))


In [ ]:
with open("all_mugs_sdf.pkl", "wb") as f:
    pickle.dump(all_sdf, f)

#Day2 : DeepSDF training to get the latent vector(z) by backprop.

In [ ]:
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:
with open("/kaggle/input/mugs-sdf-data/all_mugs_sdf.pkl", "rb") as f:
    all_sdf = pickle.load(f)

object_names = sorted(all_sdf.keys())
num_objects = len(object_names)

print("Objects:", object_names)
print("Number of objects:", num_objects)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
LATENT_DIM = 64

latent_codes = nn.Embedding(num_objects, LATENT_DIM).to(device)
nn.init.normal_(latent_codes.weight, mean=0.0, std=0.01)

In [ ]:
class DeepSDF(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.fc1 = nn.Linear(latent_dim + 3, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 1)

    def forward(self, x, z):
        h = torch.cat([x, z], dim=1)
        h = F.relu(self.fc1(h))
        h = F.relu(self.fc2(h))
        h = F.relu(self.fc3(h))
        return self.fc4(h)

In [ ]:
model = DeepSDF(LATENT_DIM).to(device)

In [ ]:
def sdf_loss(pred, gt):
    return torch.mean(torch.abs(pred - gt))

In [ ]:
#L2 regularization
def latent_reg(z):
    return torch.mean(z.pow(2))

In [ ]:
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(latent_codes.parameters()),
    lr=1e-4
)

In [ ]:
EPOCHS = 150
SAMPLES_PER_OBJECT = 2048
LAMBDA_LATENT = 1e-4

In [ ]:
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=EPOCHS,
    eta_min=1e-6
)

In [ ]:
for epoch in range(EPOCHS):
    total_loss = 0.0
    total_recon = 0.0
    total_reg = 0.0

    for obj_idx, name in enumerate(object_names):
        data = all_sdf[name]

        # Load data
        points = torch.tensor(
            data["points"], dtype=torch.float32, device=device
        )
        sdf_gt = torch.tensor(
            data["sdf"], dtype=torch.float32, device=device
        ).unsqueeze(1)

        # Random subsampling
        idx = torch.randperm(len(points))[:SAMPLES_PER_OBJECT]
        points = points[idx]
        sdf_gt = sdf_gt[idx]

        # Latent code for this object
        z = latent_codes(
            torch.tensor([obj_idx], device=device)
        ).repeat(len(points), 1)

        # Forward pass
        pred_sdf = model(points, z)

        # Reconstruction loss (L1)
        recon_loss = torch.mean(torch.abs(pred_sdf - sdf_gt))

        # Latent regularization (L2)
        reg_loss = torch.mean(z.pow(2))

        # Total loss
        loss = recon_loss + LAMBDA_LATENT * reg_loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate losses
        total_loss += loss.item()
        total_recon += recon_loss.item()
        total_reg += reg_loss.item()

    # Update learning rate (cosine annealing)
    scheduler.step()

    # Logging
    if epoch % 10 == 0 or epoch == EPOCHS - 1:
        print(
            f"Epoch {epoch:03d} | "
            f"Total Loss: {total_loss:.4f} | "
            f"Recon: {total_recon:.4f} | "
            f"Latent Reg: {total_reg:.4f} | "
            f"LR: {scheduler.get_last_lr()[0]:.6f}"
        )


In [ ]:
# Save trained DeepSDF model and latent codes
checkpoint = {
    "model_state_dict": model.state_dict(),
    "latent_codes_state_dict": latent_codes.state_dict(),
    "latent_dim": LATENT_DIM,
    "num_objects": num_objects,
    "object_names": object_names
}

torch.save(checkpoint, "/kaggle/working/deepsdf_day2_checkpoint.pth")

print("Model saved to /kaggle/working/deepsdf_day2_checkpoint.pth")


In [ ]:
# Pick one object
obj_idx = 0
name = object_names[obj_idx]
data = all_sdf[name]

# Random test points
pts = torch.tensor(
    data["points"][:5000], dtype=torch.float32, device=device
)

# Latent code
z = latent_codes(
    torch.tensor([obj_idx], device=device)
).repeat(len(pts), 1)

# Predict SDF
with torch.no_grad():
    pred_sdf = model(pts, z).cpu().numpy()


In [ ]:
import matplotlib.pyplot as plt

plt.hist(data["sdf"][:5000], bins=100, alpha=0.5, label="GT")
plt.hist(pred_sdf.flatten(), bins=100, alpha=0.5, label="Pred")
plt.legend()
plt.title("SDF Distribution Check")
plt.show()


In [ ]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

LATENT_DIM = 64  # must match training

class DeepSDF(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()

        self.fc1 = nn.Linear(3 + latent_dim, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 1)

        self.relu = nn.ReLU()

    def forward(self, x, z):
        x = torch.cat([x, z], dim=1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [ ]:
checkpoint = torch.load(
    "/kaggle/input/checkpoint/deepsdf_day2_checkpoint (1).pth",
    map_location=device
)

NUM_OBJECTS = checkpoint["num_objects"]
object_names = checkpoint["object_names"]
LATENT_DIM = checkpoint["latent_dim"]

In [ ]:
model = DeepSDF(LATENT_DIM).to(device)
latent_codes = nn.Embedding(NUM_OBJECTS, LATENT_DIM).to(device)

In [ ]:
model.load_state_dict(checkpoint["model_state_dict"])
latent_codes.load_state_dict(checkpoint["latent_codes_state_dict"])

model.eval()
latent_codes.eval()

print("Checkpoint loaded successfully")

In [ ]:
import torch
import numpy as np
import trimesh
import scipy.ndimage
from skimage import measure


In [ ]:
# Pick object index
obj_idx = 0
obj_name = object_names[obj_idx]
print("Working on:", obj_name)

# Load GT mesh just for bounds (NOT for training)
gt_mesh = trimesh.load(
    "/kaggle/input/mugs-normalised/mugs_normalized/mug1.obj",
    force="mesh"
)

bounds = gt_mesh.bounds.copy()

# Add small padding
padding = 0.05
bounds[0] -= padding
bounds[1] += padding


In [ ]:
RES_COARSE = 80

xs = np.linspace(bounds[0][0], bounds[1][0], RES_COARSE)
ys = np.linspace(bounds[0][1], bounds[1][1], RES_COARSE)
zs = np.linspace(bounds[0][2], bounds[1][2], RES_COARSE)

grid = np.stack(np.meshgrid(xs, ys, zs, indexing="ij"), -1)
coarse_points = grid.reshape(-1, 3)

In [ ]:
with torch.no_grad():
    z = latent_codes(
        torch.tensor([obj_idx], device=device)
    ).repeat(len(coarse_points), 1)

    sdf_coarse = model(
        torch.tensor(coarse_points, dtype=torch.float32, device=device),
        z
    ).cpu().numpy().squeeze()

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

obj_idx = object_names.index("mug1.obj")

pts = torch.tensor(points_flat, dtype=torch.float32, device=device)

with torch.no_grad():
    z = latent_codes(
        torch.tensor([obj_idx], device=device)
    ).repeat(len(pts), 1)

    sdf_pred = model(pts, z).cpu().numpy()


In [ ]:
SURFACE_THRESH = 0.02

mask = np.abs(sdf_coarse) < SURFACE_THRESH
surface_points = coarse_points[mask]

print("Near-surface points:", len(surface_points))

In [ ]:
noise = np.random.normal(scale=0.01, size=surface_points.shape)
refined_points = surface_points + noise

final_points = np.vstack([surface_points, refined_points])

In [ ]:
min_b = final_points.min(axis=0) - 0.02
max_b = final_points.max(axis=0) + 0.02

In [ ]:
RES_FINE = 160

xs = np.linspace(min_b[0], max_b[0], RES_FINE)
ys = np.linspace(min_b[1], max_b[1], RES_FINE)
zs = np.linspace(min_b[2], max_b[2], RES_FINE)

grid = np.stack(np.meshgrid(xs, ys, zs, indexing="ij"), -1)
fine_points = grid.reshape(-1, 3)

In [ ]:
with torch.no_grad():
    z = latent_codes(
        torch.tensor([obj_idx], device=device)
    ).repeat(len(fine_points), 1)

    sdf_fine = model(
        torch.tensor(fine_points, dtype=torch.float32, device=device),
        z
    ).cpu().numpy().squeeze()

In [ ]:
sdf_grid = sdf_fine.reshape(RES_FINE, RES_FINE, RES_FINE)

In [ ]:
sdf_grid = np.clip(sdf_grid, -0.03, 0.03)
sdf_grid = scipy.ndimage.gaussian_filter(sdf_grid, sigma=1.2)

In [ ]:
verts, faces, normals, _ = measure.marching_cubes(
    sdf_grid,
    level=0.0,
    spacing=(
        xs[1] - xs[0],
        ys[1] - ys[0],
        zs[1] - zs[0]
    )
)

In [ ]:
mesh = trimesh.Trimesh(
    vertices=verts,
    faces=faces,
    vertex_normals=normals,
    process=False
)

mesh.remove_unreferenced_vertices()
mesh.process(validate=True)
mesh.fill_holes()

mesh.show()

In [ ]:
mesh.export("/kaggle/working/sample_mug_recon.obj");